If you need to install sentinelsat

In [ ]:
!pip install sentinelsat

In [21]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import os
import geopandas as gpd
import zipfile
import getpass

In [6]:
user = 'ks04'
pword = getpass.getpass('Earthdata Login password: ')

Earthdata Login password: ········


In [8]:
#login into api
api = SentinelAPI(user, pword, 'https://scihub.copernicus.eu/dhus')

In [9]:
start_date = '20190101'
end_date = '20200331'

In [22]:
reef_footprint = geojson_to_wkt(read_geojson('Moce.geojson'))

In [24]:
#query sentinel sat api
products = api.query(reef_footprint,date = (start_date, end_date),platformname = 'Sentinel-2'\
                     ,area_relation = 'Intersects',processinglevel = 'Level-2A',\
                     order_by = 'cloudcoverpercentage')


In [33]:
# convert to Pandas DataFrame
products_df = api.to_dataframe(products)
products_df.head().T

,02aa0820-3a68-4ec1-9972-21274b8804fb,04ab6ef5-7ea6-4b43-be40-457e1169a0b9,050199d5-4e21-4f38-aef3-ffee28af4fd3,05d5a7ca-f0c3-4a24-9403-904e93d9906a,081f284e-d4a8-46f9-b0a5-c6f9f12236fc
title,S2B_MSIL2A_20200101T220919_N0213_R129_T01KCV_2...,S2A_MSIL2A_20190928T220921_N0213_R129_T01KCV_2...,S2A_MSIL2A_20190322T220921_N0211_R129_T01KCV_2...,S2B_MSIL2A_20190814T220929_N0213_R129_T01KCV_2...,S2B_MSIL2A_20191202T220919_N0213_R129_T01KCV_2...
link,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_alternative,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
link_icon,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...
summary,"Date: 2020-01-01T22:09:19.024Z, Instrument: MS...","Date: 2019-09-28T22:09:21.024Z, Instrument: MS...","Date: 2019-03-22T22:09:21.024Z, Instrument: MS...","Date: 2019-08-14T22:09:29.024Z, Instrument: MS...","Date: 2019-12-02T22:09:19.024Z, Instrument: MS..."
beginposition,2020-01-01 22:09:19.024000,2019-09-28 22:09:21.024000,2019-03-22 22:09:21.024000,2019-08-14 22:09:29.024000,2019-12-02 22:09:19.024000
endposition,2020-01-01 22:09:19.024000,2019-09-28 22:09:21.024000,2019-03-22 22:09:21.024000,2019-08-14 22:09:29.024000,2019-12-02 22:09:19.024000
ingestiondate,2020-01-02 02:21:44.385000,2019-09-29 04:24:04.919000,2019-03-23 04:00:07.459000,2019-08-15 10:42:33.829000,2019-12-03 02:08:57.908000
orbitnumber,14741,22291,19574,12739,14312
relativeorbitnumber,129,129,129,129,129


In [30]:
#creating folder for saving sentinel images
sentinel_path = 'SAFE files'
if not os.path.exists(sentinel_path):
    os.makedirs(sentinel_path)

In [31]:
#downloading num_images
num_images = 1
for i,x in enumerate(products.items()):
    k,v = x[0],x[1]
    if i < num_images:
        api.download(k, directory_path = sentinel_path)

Downloading: 100%|██████████| 775M/775M [02:28<00:00, 5.23MB/s]
MD5 checksumming: 100%|██████████| 775M/775M [00:02<00:00, 310MB/s]


In [32]:
#unzipping files
for file in os.listdir(sentinel_path):
    if file.endswith('.zip'):
        file_path = os.path.join(sentinel_path, file)
        out_path = os.path.join(sentinel_path, file.split('.')[0])

        if os.path.exists(file_path) and not os.path.exists(out_path):
            with zipfile.ZipFile(file_path,"r") as zip_ref:
                zip_ref.extractall(sentinel_path)
            os.remove(file_path)